In [8]:
import rasterio
import numpy as np
import geopandas as gpd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns


In [9]:
import rasterio

# File paths
file_paths = [
   r"D:\Jupyter Notebook\New folder\fcc.tif",
   r"D:\Jupyter Notebook\New folder\MinNDVI.tif",
   r"D:\Jupyter Notebook\New folder\MinMNDWI.tif",
   r"D:\Jupyter Notebook\New folder\MaxNDVI.tif",
   r"D:\Jupyter Notebook\New folder\MaxMNDWI.tif",
  
   r"D:\Jupyter Notebook\New folder\nighttime.tif",
   r"D:\Jupyter Notebook\New folder\elevation_trimmed.tif",
   r"D:\Jupyter Notebook\New folder\slope_trimmed.tif"
]

# Check the number of bands in each file
for file_path in file_paths:
    try:
        with rasterio.open(file_path) as src:
            print(f"File: {file_path}")
            print(f"Number of bands: {src.count}")
            print("-" * 50)
    except Exception as e:
        print(f"Error reading file: {file_path}")
        print(f"Error: {e}")
        print("-" * 50)


File: D:\Jupyter Notebook\New folder\fcc.tif
Number of bands: 11
--------------------------------------------------
File: D:\Jupyter Notebook\New folder\MinNDVI.tif
Number of bands: 1
--------------------------------------------------
File: D:\Jupyter Notebook\New folder\MinMNDWI.tif
Number of bands: 1
--------------------------------------------------
File: D:\Jupyter Notebook\New folder\MaxNDVI.tif
Number of bands: 1
--------------------------------------------------
File: D:\Jupyter Notebook\New folder\MaxMNDWI.tif
Number of bands: 1
--------------------------------------------------
File: D:\Jupyter Notebook\New folder\nighttime.tif
Number of bands: 1
--------------------------------------------------
File: D:\Jupyter Notebook\New folder\elevation_trimmed.tif
Number of bands: 1
--------------------------------------------------
File: D:\Jupyter Notebook\New folder\slope_trimmed.tif
Number of bands: 1
--------------------------------------------------


In [10]:
import rasterio
import numpy as np
import geopandas as gpd

# File paths for Sentinel and other raster files
# File paths (replace with correct paths)
sentinel_image_path =r"D:\Jupyter Notebook\New folder\fcc.tif"
min_ndvi_path = r"D:\Jupyter Notebook\New folder\MinNDVI.tif"
min_ndwi_path = r"D:\Jupyter Notebook\New folder\MinMNDWI.tif"
max_ndvi_path = r"D:\Jupyter Notebook\New folder\MaxNDVI.tif"
max_ndwi_path = r"D:\Jupyter Notebook\New folder\MaxMNDWI.tif"
nighttime_light_path = r"D:\Jupyter Notebook\New folder\nighttime.tif"
elevation_path = r"D:\Jupyter Notebook\New folder\elevation_trimmed.tif"
slope_path = r"D:\Jupyter Notebook\New folder\slope_trimmed.tif"

# Raster file paths for other layers
raster_files_paths = [
    min_ndvi_path, min_ndwi_path, max_ndvi_path, max_ndwi_path, 
    nighttime_light_path, elevation_path, slope_path
]

# Open Sentinel image and extract bands
with rasterio.open(sentinel_image_path) as sentinel_image:
    sentinel_bands = [sentinel_image.read(band) for band in [2, 3, 4, 5, 8, 11]]
    transform = sentinel_image.transform

# Open all raster files using rasterio
raster_files = []
for path in raster_files_paths:
    raster = rasterio.open(path)  # Open the raster outside the 'with' block
    raster_files.append(raster)

# Load polygons (replace with your actual shapefile path)
polygons = gpd.read_file(r"D:\Jupyter Notebook\merged_file.geojson")

# Extract centroids from the geometry
centroids = polygons.geometry.centroid
centroid_coords = [(point.x, point.y) for point in centroids]

# Function to extract raster values for coordinates
def extract_raster_values_fast(coords, raster_files, sentinel_bands=None, transform=None):
    raster_values = []
    
    # Extract raster values from raster files
    for raster in raster_files:
        values = [val[0] for val in raster.sample(coords)]
        raster_values.append(values)
    
    # If Sentinel bands are provided, append them as well
    if sentinel_bands is not None:
        for band in sentinel_bands:
            values = []
            for coord in coords:
                # Convert geographic coordinates (lon, lat) to pixel coordinates (row, col)
                row, col = ~transform * (coord[0], coord[1])
                row, col = int(row), int(col)
                
                if 0 <= row < band.shape[0] and 0 <= col < band.shape[1]:
                    values.append(band[row, col])
                else:
                    values.append(np.nan)  # Handle out-of-bounds
            raster_values.append(values)
   
    # Stack all raster values into a single array
    return np.stack(raster_values, axis=-1)

# Extract raster values for centroids
X_centroid = extract_raster_values_fast(centroid_coords, raster_files, sentinel_bands, transform)

# Flatten the features and labels
X_flat_centroid = X_centroid.reshape(-1, X_centroid.shape[-1])
y_flat_centroid = polygons["CLASS"].values.flatten()  # Replace with the correct label column

# Handle missing values (e.g., NaNs)
mask = np.isnan(X_flat_centroid).any(axis=1)
X_flat_centroid = X_flat_centroid[~mask]
y_flat_centroid = y_flat_centroid[~mask]

# Close the raster files after extraction
def close_rasters(raster_files):
    for raster in raster_files:
        raster.close()

# Close the raster files after extraction
close_rasters(raster_files)

# Continue with model training (Random Forest or any other model)


C:\Users\sahil\AppData\Local\Temp\ipykernel_22740\3549858966.py:37: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroids = polygons.geometry.centroid


In [11]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
import numpy as np

# Train-test split
X_train, X_val, y_train, y_val = train_test_split(X_flat_centroid, y_flat_centroid, test_size=0.3, random_state=42)

# Create and train the model with the correct number of features
lgb_model = lgb.LGBMClassifier(
    objective="multiclass", 
    num_class=len(np.unique(y_flat_centroid)), 
    boosting_type="gbdt", 
    num_leaves=31, 
    max_depth=-1, 
    learning_rate=0.05, 
    n_estimators=100
)

# Train the model
lgb_model.fit(X_train, y_train)

# Save the model
model_file_path = r"D:\Jupyter Notebook\Model\lightgbm.txt"  # Replace with the path where you want to save the model
lgb_model.booster_.save_model(model_file_path)

print(f"Model saved to {model_file_path}")



[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000232 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3187
[LightGBM] [Info] Number of data points in the train set: 1054, number of used features: 13
[LightGBM] [Info] Start training from score -2.345227
[LightGBM] [Info] Start training from score -2.181224
[LightGBM] [Info] Start training from score -1.407388
[LightGBM] [Info] Start training from score -1.897753
[LightGBM] [Info] Start training from score -2.578321
[LightGBM] [Info] Start training from score -1.963135
[LightGBM] [Info] Start training from score -1.713324
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further sp

In [12]:
# Predict on the validation set
y_pred = lgb_model.predict(X_val)

# Calculate accuracy
accuracy = accuracy_score(y_val, y_pred)
print("Validation Accuracy:", accuracy)

# Classification report
print("Classification Report:\n", classification_report(y_val, y_pred))



Validation Accuracy: 0.9514348785871964
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        33
           1       1.00      1.00      1.00        40
           2       0.97      0.97      0.97       122
           3       0.95      0.90      0.93        83
           4       0.98      1.00      0.99        40
           5       0.90      0.90      0.90        63
           6       0.91      0.94      0.93        72

    accuracy                           0.95       453
   macro avg       0.96      0.96      0.96       453
weighted avg       0.95      0.95      0.95       453



In [13]:
# Import necessary libraries
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import joblib

# Class labels
class_labels = [
    "Waterbodies",    # Class 0
    "Forest",         # Class 1
    "Built-up",       # Class 2
    "Barren Land",    # Class 3
    "ShrubLand",      # Class 4
    "Fallow Land",    # Class 5
    "CropLand"        # Class 6
]

# Step 1: Compute confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Step 2: Visualize the confusion matrix
plt.figure(figsize=(10, 7))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
plt.title('LightGBM Confusion Matrix')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.tight_layout()  # Adjust layout
plt.savefig("confusion_matrix(LightGbm.png")  # Save the confusion matrix
plt.show()

# Step 3: Calculate User Accuracy (UA) and Producer Accuracy (PA)
ua = cm.diagonal() / cm.sum(axis=0)  # User Accuracy
pa = cm.diagonal() / cm.sum(axis=1)  # Producer Accuracy

# Step 4: Print the User Accuracy (UA) and Producer Accuracy (PA) values
print("User Accuracy (UA):")
for i, accuracy in enumerate(ua):
    print(f"{class_labels[i]}: {accuracy:.2f}")  # Print each class's UA with 2 decimal places

print("\nProducer Accuracy (PA):")
for i, accuracy in enumerate(pa):
    print(f"{class_labels[i]}: {accuracy:.2f}")  # Print each class's PA with 2 decimal places

# Step 5: Plot User Accuracy (UA) and Producer Accuracy (PA) in the same graph
x = range(len(class_labels))  # Indices for classes
plt.figure(figsize=(12, 6))
bars_ua = plt.bar(x, ua, width=0.4, label="User Accuracy (UA)", color="skyblue", align="center")
bars_pa = plt.bar([i + 0.4 for i in x], pa, width=0.4, label="Producer Accuracy (PA)", color="lightgreen", align="center")
plt.xticks([i + 0.2 for i in x], class_labels, rotation=45)  # Center tick labels
plt.title('User Accuracy (UA) and Producer Accuracy (PA) of LightGBM')
plt.xlabel('Classes')
plt.ylabel('Accuracy')
plt.ylim(0, 1)  # Accuracy ranges from 0 to 1
plt.legend(loc="upper left")

# Adjust layout
plt.tight_layout()  # Adjust layout
plt.savefig("ua_pa_graph(LightGbm.png")  # Save the UA and PA graph
plt.show()


NameError: name 'y_test' is not defined

In [ ]:
import numpy as np
import rasterio
import lightgbm as lgb

def read_and_resize_rasters(raster_files, sentinel_composite_file, selected_bands, window_size=128):
    # Initialize an empty list to store all rasters
    all_rasters = []

    # Read and resize the raster files (non-Sentinel)
    for raster_file in raster_files:
        with rasterio.open(raster_file) as src:
            raster_data = src.read(1)  # Read the first band (assuming single-band rasters)
            all_rasters.append(raster_data)
    
    # Read the selected bands from the composite Sentinel image
    with rasterio.open(sentinel_composite_file) as src:
        for band in selected_bands:  # Iterate through the selected bands
            sentinel_band_data = src.read(band)  # Read each selected band
            all_rasters.append(sentinel_band_data)

    # Convert the list of rasters into a single numpy array (shape will be [height, width, num_bands])
    all_rasters_array = np.stack(all_rasters, axis=-1)
    
    return all_rasters_array

def predict_classes_in_windows(raster_files, sentinel_composite_file, selected_bands, model, window_size=128):
    # Read rasters and selected Sentinel bands
    input_data = read_and_resize_rasters(raster_files, sentinel_composite_file, selected_bands, window_size)
    
    # Get the image dimensions
    height, width, _ = input_data.shape
    
    # Check that we have the correct number of features (13)
    print("Shape of input data:", input_data.shape)  # Should be (height, width, 13)
    
    # Initialize an empty array to store the predicted classes
    all_predictions = np.zeros((height, width))  # Initialize an empty array to store predictions

    # Process the image in smaller chunks or windows
    for y in range(0, height, window_size):
        for x in range(0, width, window_size):
            # Define the window boundaries, ensuring it doesn't exceed image dimensions
            window = input_data[y:min(y+window_size, height), x:min(x+window_size, width), :]
            
            # Flatten the window along the band axis (keeping only the 13 features)
            window_flat = window.reshape(-1, 13)  # Flatten only the band dimension
            
            # Check the shape before prediction
            print("Shape of window_flat:", window_flat.shape)  # Should be (num_pixels_in_window, 13)
            
            # Predict using the model (assuming it's a classification model with 7 classes)
            prediction = model.predict(window_flat)
            
            # Debugging: print the shape of the prediction
            print(f"Shape of prediction: {prediction.shape}")
            
            # If the model is returning class probabilities, take the argmax to get the predicted class
            predicted_classes = np.argmax(prediction, axis=1)  # Class with highest probability
            
            # Ensure prediction is of the correct size, reshape it to match the window size (e.g., 128x128)
            predicted_window = predicted_classes.reshape(min(window_size, height - y), min(window_size, width - x))
            
            # Store the predictions into the corresponding section of the result array
            all_predictions[y:y + predicted_window.shape[0], x:x + predicted_window.shape[1]] = predicted_window
    
    return all_predictions

# Example usage
raster_files = [
    r"D:\Jupyter Notebook\New folder\MinNDVI.tif",
    r"D:\Jupyter Notebook\New folder\MinMNDWI.tif",
    r"D:\Jupyter Notebook\New folder\MaxNDVI.tif",
    r"D:\Jupyter Notebook\New folder\MaxMNDWI.tif",
   
    r"D:\Jupyter Notebook\New folder\nighttime.tif",
    r"D:\Jupyter Notebook\New folder\elevation_trimmed.tif",
    r"D:\Jupyter Notebook\New folder\slope_trimmed.tif"
]

sentinel_composite_file = r"D:\Jupyter Notebook\New folder\fcc.tif"
selected_bands = [2, 3, 4, 5, 8, 11]

# Load the LightGBM model
model = lgb.Booster(model_file=r"D:\Jupyter Notebook\Model\LightGbm.txt")

# Perform prediction for the entire image
predicted_classes = predict_classes_in_windows(
    raster_files=raster_files,
    sentinel_composite_file=sentinel_composite_file,
    selected_bands=selected_bands,
    model=model
)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import ListedColormap, BoundaryNorm

def visualize_predictions_with_colormap(predictions, colormap, title="Predicted Classes Visualization"):
    """
    Visualizes the predicted classes as an image with a specified colormap.
    
    Parameters:
    - predictions: 2D numpy array of predicted classes.
    - colormap: Dictionary mapping class IDs to RGB tuples.
    - title: Title of the visualization.
    """
    # Convert RGB tuples to normalized [0, 1] range for Matplotlib
    colors = [(r / 255, g / 255, b / 255) for r, g, b in colormap.values()]
    cmap = ListedColormap(colors)
    norm = BoundaryNorm(np.arange(-0.5, len(colormap) + 0.5), len(colormap))

    plt.figure(figsize=(10, 10))
    plt.imshow(predictions, cmap=cmap, norm=norm, interpolation='nearest')
    cbar = plt.colorbar(ticks=range(len(colormap)), fraction=0.046, pad=0.04)
    cbar.ax.set_yticklabels([f"Class {i}" for i in colormap.keys()])
    plt.title(title, fontsize=16)
    plt.xlabel("Width", fontsize=12)
    plt.ylabel("Height", fontsize=12)
    plt.show()

# Define the colormap for the classes
colormap = {
    0: (115, 223, 255),  # 73DFFF
    1: (34, 168, 0),     # 22A800
    2: (202, 0, 0),      # CA0000
    3: (210, 180, 140),  # D2B48C
    4: (128, 128, 0),    # 808000
    5: (230, 230, 0),    # E6E600
    6: (255, 255, 190)   # FFFFBE
}

# Assuming `predicted_classes` is reshaped to match the original raster dimensions
visualize_predictions_with_colormap(predicted_classes, colormap, title=" LightGbm Predicted Classes Across the Image")


In [ ]:
import rasterio
from rasterio.transform import from_origin

def save_predictions_as_tiff(predictions, reference_raster_path, output_tiff_path):
    """
    Save the predicted classes as a TIFF file using the metadata of a reference raster.

    Parameters:
    - predictions: 2D numpy array of predicted classes.
    - reference_raster_path: Path to the reference raster file (for metadata like transform and CRS).
    - output_tiff_path: Path to save the output TIFF file.
    """
    with rasterio.open(reference_raster_path) as src:
        # Extract metadata from the reference raster
        transform = src.transform
        crs = src.crs
        dtype = 'uint8'  # Assuming class labels are integers between 0-255

    # Define metadata for the output TIFF
    height, width = predictions.shape
    meta = {
        'driver': 'GTiff',
        'height': height,
        'width': width,
        'count': 1,  # Single band
        'dtype': dtype,
        'crs': crs,
        'transform': transform
    }

    # Write the predictions to the TIFF file
    with rasterio.open(output_tiff_path, 'w', **meta) as dst:
        dst.write(predictions.astype(dtype), 1)

# Example usage
reference_raster_path = r"C:\Users\sahil\OneDrive - Bharati Vidyapeeth\Desktop\New folder (5)\MinNDVI.tif"  # Replace with your reference raster
output_tiff_path = r"D:\Jupyter Notebook\Tiff File\lightgbm.tif" # Replace with your desired output path
# Assuming `predicted_classes` is reshaped to match the original raster dimensions
save_predictions_as_tiff(predicted_classes, reference_raster_path, output_tiff_path)
print(f"Predictions saved as TIFF file: {output_tiff_path}")


In [ ]:
import folium
import rasterio
import numpy as np
from folium.raster_layers import ImageOverlay
from folium.plugins import FloatImage

# Load your classified raster image
raster_path = r"D:\Jupyter Notebook\Tiff File\lightgbm.tif" # Replace with your GeoTIFF file path
with rasterio.open(raster_path) as src:
    raster_data = src.read(1)  # Read the first band
    bounds = src.bounds  # Get the bounds of the raster
    transform = src.transform

# Define your class-to-color mapping (RGB values for each class)
class_colors = {
    0: [115, 223, 255],  # Waterbodies
    1: [34, 168, 0],     # Forest
    2: [202, 0, 0],      # Built-up
    3: [210, 180, 140],  # Barren Land
    4: [128, 128, 0],    # Shrubland
    5: [230, 230, 0],    # Fallow Land
    6: [255, 255, 190],  # Cropland
}

# Convert raster data to an RGB image using the class colors
rgb_image = np.zeros((raster_data.shape[0], raster_data.shape[1], 3), dtype=np.uint8)

for class_value, color in class_colors.items():
    mask = raster_data == class_value  # Find pixels for the current class
    rgb_image[mask] = color  # Assign the RGB color to those pixels

# Save the RGB image as a PNG (optional step for debugging)
import matplotlib.pyplot as plt
plt.imsave("classified_image.png", rgb_image)

# Define the map bounds (SouthWest and NorthEast corners)
map_bounds = [[bounds.bottom, bounds.left], [bounds.top, bounds.right]]

# Create a Folium map centered on your raster
m = folium.Map(location=[(bounds.top + bounds.bottom) / 2, (bounds.left + bounds.right) / 2],
               zoom_start=10, tiles='OpenStreetMap')

# Add the Google Satellite layer
folium.TileLayer('https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}', 
                 attr='Google Satellite', name='Google Satellite').add_to(m)

# Add your classified image as an overlay
ImageOverlay(
    image="classified_image.png",  # Use the saved PNG
    bounds=map_bounds,
    opacity=0.7
).add_to(m)

# Add a layer control for toggling layers
folium.LayerControl().add_to(m)

# Add a title to the map
title_html = '''
<div style="position: fixed; 
            top: 10px; left: 50%; transform: translate(-50%, 0);
            z-index: 1000;
            background-color: white;
            padding: 10px;
            font-size: 20px;
            font-weight: bold;
            border: 2px solid black;
            border-radius: 5px;">
    LIGHTGBM - Land Use/Land Cover Classification
</div>
'''
m.get_root().html.add_child(folium.Element(title_html))

# Add a legend to the map
legend_html = '''
<div style="position: fixed; 
            bottom: 20px; left: 20px; width: 200px; height: 325px; 
            z-index: 1000;
            background-color: white;
            border: 2px solid black;
            border-radius: 5px;
            padding: 10px;">
    <h4 style="margin-top: 0; text-align: center;">Legend</h4>
    <div style="display: flex; align-items: center;"><div style="width: 40px; height: 40px; background-color: rgb(115, 223, 255); margin-right: 10px;"></div>Waterbodies</div>
    <div style="display: flex; align-items: center;"><div style="width: 40px; height: 40px; background-color: rgb(34, 168, 0); margin-right: 10px;"></div>Forest</div>
    <div style="display: flex; align-items: center;"><div style="width: 40px; height: 40px; background-color: rgb(202, 0, 0); margin-right: 10px;"></div>Built-up</div>
    <div style="display: flex; align-items: center;"><div style="width: 40px; height: 40px; background-color: rgb(210, 180, 140); margin-right: 10px;"></div>Barren Land</div>
    <div style="display: flex; align-items: center;"><div style="width: 40px; height: 40px; background-color: rgb(128, 128, 0); margin-right: 10px;"></div>Shrubland</div>
    <div style="display: flex; align-items: center;"><div style="width: 40px; height: 40px; background-color: rgb(230, 230, 0); margin-right: 10px;"></div>Fallow Land</div>
    <div style="display: flex; align-items: center;"><div style="width: 40px; height: 40px; background-color: rgb(255, 255, 190); margin-right: 10px;"></div>Cropland</div>
</div>
'''
m.get_root().html.add_child(folium.Element(legend_html))

# Display the map
m
